In [1]:
# TensorFlow 라이브러리를 설치하는 명령어
# 느낌표 (!)는 Jupyter Notebook이나 Colab 환경에서 셸 명령어를 실행할 때 사용됨
# pip은 Python 패키지 관리자이며, TensorFlow를 현재 Python 환경에 설치함
!pip install tensorflow


In [2]:
# TensorFlow 라이브러리를 tf라는 이름(별칭)으로 불러옴
import tensorflow as tf

# TensorFlow의 Keras API에서 MNIST 데이터셋 불러오기
# MNIST: 0~9까지 손글씨 숫자 이미지 (28x28 픽셀, 흑백)
mnist = tf.keras.datasets.mnist

# MNIST 데이터를 학습용(train)과 테스트용(test)으로 나누어 로드
# x_train, y_train : 학습용 이미지와 해당 라벨(정답 숫자)
# x_test, y_test   : 테스트용 이미지와 해당 라벨
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 학습 데이터의 형태(shape) 출력
# x_train.shape → (60000, 28, 28): 6만 장의 28x28 픽셀 이미지
# y_train.shape → (60000,): 각 이미지에 대한 정답 숫자(0~9)
print(x_train.shape, y_train.shape)

# 테스트 데이터의 형태(shape) 출력
# x_test.shape → (10000, 28, 28): 1만 장의 테스트 이미지
# y_test.shape → (10000,): 테스트 이미지의 정답 숫자
print(x_test.shape, y_test.shape)


(60000, 28, 28) (60000,)
(10000, 28, 28) (10000,)


In [3]:
# 이미지 픽셀 값을 0~1 사이로 정규화(Normalization)
# 원래 MNIST의 픽셀 값은 0~255(8비트 흑백 이미지)
# 신경망 학습 시 입력값의 범위를 0~1로 맞추면
# 학습이 안정되고 빠르게 수렴하는 효과가 있음

x_train, x_test = x_train / 255.0, x_test / 255.0


In [4]:
# Keras의 Sequential 모델을 이용해 신경망 모델을 정의
# Sequential 모델은 층(layer)을 순차적으로 쌓은 형태로 구성됨
mip_model = tf.keras.models.Sequential([
    
    # 입력층: 28x28 크기의 이미지를 1차원 배열로 변환 (Flatten)
    # input_shape=(28,28): 입력 데이터는 28x28 픽셀 크기의 이미지
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    
    # 은닉층: 128개의 뉴런을 가진 fully-connected (dense) 층
    # 활성화 함수로 ReLU (Rectified Linear Unit)를 사용
    # ReLU는 음수를 0으로 변환하고, 양수는 그대로 유지하는 함수로, 비선형성을 부여
    tf.keras.layers.Dense(128, activation='relu'),
    
    # 출력층: 10개의 뉴런을 가진 출력층
    # 각 뉴런은 0~9까지의 클래스에 대한 확률을 출력
    # 활성화 함수로 Softmax를 사용, 각 출력값은 0~1 사이의 확률 값
    # Softmax는 다중 분류 문제에서 사용됨
    tf.keras.layers.Dense(10, activation='softmax')
])


C:\Users\COMPUTER\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# 모델을 학습할 준비를 완료하는 과정
# 1. optimizer: 최적화 알고리즘을 설정
# 2. loss: 손실 함수(모델의 예측이 실제 값과 얼마나 차이가 나는지 측정)
# 3. metrics: 모델 평가에 사용할 지표

mip_model.compile(
    optimizer='adam',  # 최적화 방법: Adam 옵티마이저 사용
    loss='sparse_categorical_crossentropy',  # 손실 함수: sparse categorical crossentropy 사용
    metrics=['accuracy']  # 평가 지표: 정확도(accuracy)를 사용
)


In [6]:
# 모델의 요약 정보 출력
# 각 층의 이름, 출력 형태, 파라미터 수 등을 확인할 수 있음
mip_model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         100,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 101,770 (397.54 KB)

 Trainable params: 101,770 (397.54 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 모델을 학습시킴
# x_train: 훈련 데이터(28x28 크기의 이미지)
# y_train: 훈련 데이터의 라벨 (0~9 숫자)
# epochs=5: 훈련 데이터를 5번 반복하여 모델을 학습시킴
# 각 epoch마다 모델의 가중치가 업데이트됨

mip_model.fit(x_train, y_train, epochs=5)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 770us/step - accuracy: 0.9256 - loss: 0.2610
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 743us/step - accuracy: 0.9661 - loss: 0.1142
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 773us/step - accuracy: 0.9758 - loss: 0.0789
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 742us/step - accuracy: 0.9826 - loss: 0.0577
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1s 745us/step - accuracy: 0.9865 - loss: 0.0439


In [8]:
# 테스트 데이터를 사용하여 모델 성능 평가
# x_test: 테스트 데이터(28x28 크기의 이미지)
# y_test: 테스트 데이터의 라벨 (0~9 숫자)
# verbose=2: 출력 형식을 설정 (간단한 정보만 출력)

mip_model.evaluate(x_test, y_test, verbose=2)


313/313 - 0s - 823us/step - accuracy: 0.9770 - loss: 0.0758


[0.07582481950521469, 0.9769999980926514]

In [9]:
# TensorFlow Keras Model 상속을 이용한 커스텀 MLP 모델 정의
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()

        # Flatten 층: 입력 이미지를 1차원 벡터로 변환
        self.flatten = tf.keras.layers.Flatten()

        # Dense 층: 128개의 뉴런, ReLU 활성화 함수
        self.dense = tf.keras.layers.Dense(128, activation='relu')

        # 출력층: 10개의 뉴런, Softmax 활성화 함수
        # 문제 있음: self.flatten 대신 Dense로 덮어쓰기 되어 있음 (오타)
        self.flatten = tf.keras.layers.Dense(10, activation='softmax')
    
    # 모델의 순전파 정의
    def call(self, inputs):
        x = self.flatten(inputs)   # 여기서 flatten은 Dense로 덮어쓰기 됨 (문제)
        x = self.dense(x)
        return self.softmax(x)     # self.softmax는 정의되지 않음 (문제)


In [11]:
# x_train과 x_test 데이터를 (28, 28, 1) 형태로 변환
# 이 데이터는 흑백 이미지로, 각 이미지는 28x28 크기이며 채널은 1개입니다 (흑백)
x_train_4d = x_train.reshape(-1, 28, 28, 1)
x_test_4d = x_test.reshape(-1, 28, 28, 1)

# CNN 모델 정의
cnn_model = tf.keras.models.Sequential([
    # 첫 번째 합성곱 층 (필터 32개, 커널 크기 3x3, 활성화 함수 ReLU)
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    
    # 첫 번째 풀링 층 (2x2 풀링)
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # 두 번째 합성곱 층 (필터 64개, 커널 크기 3x3, 활성화 함수 ReLU)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    
    # 두 번째 풀링 층 (2x2 풀링)
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # 세 번째 합성곱 층 (필터 64개, 커널 크기 3x3, 활성화 함수 ReLU)
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    
    # 출력층을 위한 Flatten (2D 텐서를 1D로 변환)
    tf.keras.layers.Flatten(),
    
    # 첫 번째 완전 연결(Dense) 층 (뉴런 64개)
    tf.keras.layers.Dense(64, activation='relu'),
    
    # 최종 출력층 (10개의 뉴런, softmax 활성화 함수)
    tf.keras.layers.Dense(10, activation='softmax')
])

# 모델 컴파일
cnn_model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

# 모델 요약 출력
cnn_model.summary()

# 모델 학습
cnn_model.fit(x_train_4d, y_train, epochs=5)
# 모델 평가
cnn_model.evaluate(x_test_4d, y_test, verbose=2)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 26, 26, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 13, 13, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 11, 11, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 5, 5, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 3, 3, 64)            │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 576)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 93,322 (364.54 KB)

 Trainable params: 93,322 (364.54 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9512 - loss: 0.1547
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9859 - loss: 0.0470
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9891 - loss: 0.0341
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9915 - loss: 0.0267
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9934 - loss: 0.0207
313/313 - 0s - 2ms/step - accuracy: 0.9915 - loss: 0.0297


[0.02971799299120903, 0.9915000200271606]

In [12]:
# x_train과 x_test를 (28, 28) 크기에서 (28, 28, 1) 형태로 변환
# reshape: 각 이미지를 1채널로 (batch_size, height, width, channels) 형태로 변환
x_train_4d = x_train.reshape(-1, 28, 28, 1)
x_test_4d = x_test.reshape(-1, 28, 28, 1)

# x_train과 x_test의 크기를 28x28에서 32x32로 리사이즈
# tf.image.resize를 사용하여 크기 변경
resized_x_train = tf.image.resize(x_train_4d, [32, 32])
resized_x_test = tf.image.resize(x_test_4d, [32, 32])

# ResNet50V2 모델 정의
# input_shape=(32,32,1): 32x32 크기의 1채널 이미지 입력
# classes=10: MNIST 데이터셋은 10개의 클래스를 갖고 있음
# weights=None: 사전 학습된 가중치를 사용하지 않음
resent_model = tf.keras.applications.ResNet50V2(
    input_shape=(32, 32, 1),
    classes=10,
    weights=None
)

# 모델 컴파일
# optimizer='adam': Adam 최적화 알고리즘 사용
# loss='sparse_categorical_crossentropy': 다중 클래스 분류에 적합한 손실 함수 사용
# metrics=['accuracy']: 정확도를 평가 지표로 설정
resent_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy',
                     metrics=['accuracy'])

# 모델 학습
# resized_x_train과 y_train을 사용하여 모델을 5번의 에폭 동안 학습
resent_model.fit(resized_x_train, y_train, epochs=5)

# 모델 평가
# 테스트 데이터를 사용하여 모델의 성능을 평가하고, 정확도를 출력
resent_model.evaluate(resized_x_test, y_test, verbose=2)


Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 360s 186ms/step - accuracy: 0.9417 - loss: 0.2147
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 381s 203ms/step - accuracy: 0.9744 - loss: 0.0980
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 382s 204ms/step - accuracy: 0.9732 - loss: 0.1079
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 381s 203ms/step - accuracy: 0.9822 - loss: 0.0694
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 384s 205ms/step - accuracy: 0.9825 - loss: 0.0688
313/313 - 14s - 44ms/step - accuracy: 0.9792 - loss: 0.0754


[0.07542168349027634, 0.979200005531311]